<a href="https://colab.research.google.com/github/LleilaA13/Thesis-MUL/blob/main/notebooks/diversity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# The Diversity Objective

This notebook demonstrates the diversity objective, which is used to generate multiple different visualizations from a single neuron or channel.

# Feature visualization w/ Diversity function:
  - it defines an img parametrization function
  - constructs an optimization objective by maximizing a specific channel in the "mixed5a" layer and by introducing a diversity loss to encourage variarion

## Install, Import, Load Model

In [ ]:
!pip install --quiet git+https://github.com/greentfrapp/lucent.git

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━ 565.7/664.8 MB 143.3 MB/s eta 0:00:01

In [ ]:
import torch

from lucent.optvis import render, param, transform, objectives
from lucent.modelzoo import inceptionv1

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = inceptionv1(pretrained=True)
_ = model.to(device).eval()

## An Example

With the diversity objective, we have to submit a batch of images for optimization. The diversity objective then tries to maximize the difference in feature representations between the images in the batch.

Specifically, the objective penalizes cosine similarity between the feature representations.

In [ ]:
batch_param_f = lambda: param.image(128, batch=4) #created 4 images of size 128x128

obj = objectives.channel("mixed5a", 9) - 1e2 * objectives.diversity("mixed5a")

_ = render.render_vis(model, obj, batch_param_f, show_inline=True)
#! this is where the input img is gradually modified to make the selected neuron fire more

# above:
This is an exampke in which we maximize activation of channel 9 in layer mixed5a, add a diversity term and the coefficient 100 balances activation strength vs. diversity

## More Examples

More examples reproducing results from the Distill article [Feature Visualization](https://distill.pub/2017/feature-visualization/).

# Below, an exmaple with more diversity:
Here the difference is that it targets channel 97 in mixed4a. instead and uses a higher diversity weight (1000), leading to even more varied outputs.

In [ ]:
# Different curvy facets

obj = objectives.channel("mixed4a", 97) - 1e3 * objectives.diversity("mixed4a") # here we use a higher weight on the diversity term

_ = render.render_vis(model, obj, batch_param_f, show_inline=True)

In [ ]:
# Different shapes with the same fur texture

obj = objectives.channel("mixed4a", 143) - 1e2 * objectives.diversity("mixed4a")

_ = render.render_vis(model, obj, batch_param_f, show_inline=True)

In [ ]:
# Cars and Cats

obj = objectives.channel("mixed4e", 55) - 1e2 * objectives.diversity("mixed4e")

_ = render.render_vis(model, obj, batch_param_f, show_inline=True)

## Try it out!

Select your favorite channel or neuron or just pick a random one! Try adjusting the weight on the diversity term to see how that makes a difference!

In [ ]:
# Flowers and err... other stuff?

batch_param_f = lambda: param.image(128, batch=4, decorrelate=False) # disable channel decorrelation for more trippy images

obj = objectives.channel("mixed4d_3x3_bottleneck_pre_relu_conv", 139) - 1e3 * objectives.diversity("mixed4d_3x3_bottleneck_pre_relu_conv")

_ = render.render_vis(model, obj, batch_param_f, show_inline=True)

## What is happening here?
This notebook is visualizing what different neurons, or channels, in INCEPTIONV1 "see".

# Define the objective: make the nw activate a specific neuron
    obj = objectives.channel("mixed5a", 9) - 1e2 * objectives.diversity("mixed5a")
        we want channel 9 to become as active as possible, thus this forces the nw to generate an image that storngly activates that specific neuron.
        objectives.diversity() encourages varied images instead of identical ones. the -1e2* means diversity is weighted 100 times less than activation.
    Without diversity, we'd get 4 almost identical images, with it instead the nw tries different ways to activate the same neuron, leading to more creative and diverse results.
